# Train Interspeech

## Data

In [1]:
import pandas as pd

# Cargar el CSV para male
data = pd.read_csv('reduced_pseudo_male.csv')
print(len(data))
# Eliminar filas con valores nulos solo en la columna 'text'
data = data.dropna(subset=['text'])
print(len(data))
data.head()

66250
66250


,FileName,text,EmoClass,EmoAct,EmoVal,EmoDom,SpkrID,Gender,Split_Set
0,MSP-PODCAST_2432_0200,... happening there as well. and you have the ...,S,2.800000,2.200000,3.40,1425,Male,Train
1,MSP-PODCAST_0133_0033,look they're - they're reporting on absolute ...,D,6.800000,2.800000,6.60,54,Male,Development
2,MSP-PODCAST_0288_0019,"mr. [excess 00:01:24] also known as ike, bbc r...",H,5.333333,5.416667,4.75,123,Male,Train
3,MSP-PODCAST_2546_0333_0003,and instead of us just handing people masks...,H,4.600000,4.000000,4.80,1644,Male,Development
4,MSP-PODCAST_3371_0004_0001,... i decided to take a trip to canada with my...,N,3.400000,4.800000,4.20,1769,Male,Development


In [2]:
# Filtrar los datos en tres conjuntos basados en la columna 'Split_Set'
train_df = data.loc[data['Split_Set'] == 'Train']
dev_df = data.loc[data['Split_Set'] == 'Development']
test_df = data.loc[data['Split_Set'] == 'Test']

# Verifica las primeras filas de cada conjunto
print("Conjunto Train:")
print(train_df.head())

print("Conjunto Development:")
print(dev_df.head())

print("Conjunto Test:")
print(test_df.head())

Conjunto Train:
                FileName                                               text  \
0  MSP-PODCAST_2432_0200  ... happening there as well. and you have the ...   
2  MSP-PODCAST_0288_0019  mr. [excess 00:01:24] also known as ike, bbc r...   
5  MSP-PODCAST_0545_0449  man, the power of contrast is so, i think, eas...   
6  MSP-PODCAST_5492_2849  ... we're older. so why not allow a little bit...   
7  MSP-PODCAST_0153_0436              just clowning in the studio, people.    

  EmoClass    EmoAct    EmoVal    EmoDom  SpkrID Gender Split_Set  
0        S  2.800000  2.200000  3.400000    1425   Male     Train  
2        H  5.333333  5.416667  4.750000     123   Male     Train  
5        N  3.200000  3.800000  3.800000     227   Male     Train  
6        A  5.400000  2.400000  5.000000    2889   Male     Train  
7        H  5.777778  5.111111  5.444444      15   Male     Train  
Conjunto Development:
                      FileName  \
1        MSP-PODCAST_0133_0033   
3   MSP-POD

In [3]:
# Establecer la semilla para garantizar reproducibilidad
seed = 42

# Seleccionar aleatoriamente las primeras N filas del conjunto de entrenamiento
train_df = train_df.sample(n=1000, random_state=seed)

# Seleccionar aleatoriamente las primeras N filas del conjunto de desarrollo
dev_df = dev_df.sample(n=250, random_state=seed)

In [5]:
num_labels = 10
id2label = {
    0: "A",
    1: "S",
    2: "H",
    3: "U",
    4: "F",
    5: "D",
    6: "C",
    7: "N",
    8: "O",
    9: "X"
}
label2id = {
    "A": 0,
    "S": 1,
    "H": 2,
    "U": 3,
    "F": 4,
    "D": 5,
    "C": 6,
    "N": 7,
    "O": 8,
    "X": 9
}

# Convertir EmoClass a valores numéricos si es necesario
train_df['EmoClass'] = train_df['EmoClass'].map(label2id).astype(int)
dev_df['EmoClass'] = dev_df['EmoClass'].map(label2id).astype(int)

In [6]:
from datasets import Dataset
from transformers import AutoTokenizer

# Modelo
model_ckpt = "distilbert-base-uncased"

# Cargar el tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# Función para tokenizar los datos
def tokenize_function(examples):
    # Verificar que estamos pasando una lista de textos
    texts = examples['text']
    return tokenizer(texts, padding="max_length", truncation=True)

# Asegurarse de que 'train_df' y 'dev_df' son objetos Dataset de Hugging Face
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)

# Tokenizamos ambos conjuntos de datos
train_dataset = train_dataset.map(tokenize_function, batched=True)
dev_dataset = dev_dataset.map(tokenize_function, batched=True)

# Renombrar columna de labels
train_dataset = train_dataset.rename_column("EmoClass", "labels")
dev_dataset = dev_dataset.rename_column("EmoClass", "labels")


/home/jggomez/Desktop/Interspeech/interspeech-nlp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 2500/2500 [00:00<00:00, 6372.67 examples/s]


## Code

In [7]:
from transformers import AutoTokenizer
import torch
print(torch.__version__)  # Esto debería mostrarte la versión de PyTorch instalada

2.5.1+cu124


In [8]:
from transformers import AutoModelForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels, id2label=id2label, label2id=label2id).to(device)

/home/jggomez/Desktop/Interspeech/interspeech-nlp/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    # Obtener reporte completo
    report = classification_report(labels, predictions, output_dict=True)
    
    # Obtener la matriz de confusión
    conf_matrix = confusion_matrix(labels, predictions)
    
    # Extraer métricas para cada clase y globales
    metrics = {
        'accuracy': report['accuracy'],
        'weighted_f1': report['weighted avg']['f1-score'],
        # 'weighted_precision': report['weighted avg']['precision'],
        # 'weighted_recall': report['weighted avg']['recall'],
        # La matriz de confusión no se incluye normalmente como una métrica devuelta porque no es un escalar
        # 'confusion_matrix': conf_matrix.tolist()  # Convertir a lista para asegurarse de que es serializable si es necesario
    }
    
    # # Añadir métricas específicas por clase si se requiere
    # for label, scores in report.items():
    #     if label not in ["accuracy", "macro avg", "weighted avg"]:
    #         metrics[f'{label}_precision'] = scores['precision']
    #         metrics[f'{label}_recall'] = scores['recall']
    #         metrics[f'{label}_f1'] = scores['f1-score']
    #         metrics[f'{label}_support'] = scores['support']
    
    return metrics

In [10]:
from transformers import Trainer, TrainingArguments

batch_size = 8
logging_steps = len(train_dataset) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(
    output_dir=model_name,
    num_train_epochs=10,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    disable_tqdm=False,
    logging_steps=logging_steps,
    #push_to_hub=True,
    log_level="error"
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer
)

/home/jggomez/Desktop/Interspeech/interspeech-nlp/.venv/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_1879510/2523129008.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
preds_output = trainer.predict(dev_dataset)
preds_output.metrics

/Users/jorgegarcelan/Desktop/UNI/6-LAB TSC/interspeech-nlp/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jorgegarcelan/Desktop/UNI/6-LAB TSC/interspeech-nlp/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jorgegarcelan/Desktop/UNI/6-LAB TSC/interspeech-nlp/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division`

{'test_loss': 2.095921516418457,
 'test_accuracy': 0.28,
 'test_weighted_f1': 0.1264516129032258,
 'test_runtime': 0.7566,
 'test_samples_per_second': 33.041,
 'test_steps_per_second': 5.286}